Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing the dataset

In [ ]:
dataset=pd.read_csv('WineQuality.csv')

Deriving Variables based on mi scores and dataset content

In [2]:
dataset['fvratio']=dataset['fixed acidity']/dataset['volatile acidity']
shift=dataset.pop('fvratio')
dataset.insert(2, 'fvratio', shift)
dataset['ftratio']=dataset['free sulfur dioxide']/dataset['total sulfur dioxide']
shift1=dataset.pop('ftratio')
dataset.insert(2, 'ftratio', shift1)
dataset['csratio']=dataset['chlorides']/dataset['sulphates']
shift2=dataset.pop('csratio')
dataset.insert(2, 'csratio', shift2)
shift3=dataset.pop('Type')
dataset.insert(3,'Type',shift3)

Seperating the Dependent and Independent Variables

In [ ]:
X=dataset.iloc[:,1:-1].values
Y=dataset.iloc[:,-1].values

Cross Checking the dataset by printing header

In [ ]:
print(dataset.head())

Check for Missing values

In [5]:
if dataset.isnull().any().any():
  print('true')

Handling Missing values

In [6]:
rows_with_nan = dataset[dataset.isnull().any(axis=1)]
print(rows_with_nan)


Empty DataFrame
Columns: [Unnamed: 0, fixed acidity, csratio, Type, ftratio, fvratio, volatile acidity, citric acid, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, density, pH, sulphates, alcohol, quality]
Index: []


In [7]:
#Since rows with Nan values are comparitively less we shall proceed by removing those
dataset = dataset.dropna()

In [8]:
if dataset.isnull().any().any():
  print('true')

Encoding the Cateogrical Variables

In [9]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])

Feature Engineering to find mi score of variables

In [10]:
from sklearn.feature_selection import mutual_info_regression
mi_scores = mutual_info_regression(X,Y)
feature_names=['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']
feature_mi_scores = list(zip(feature_names, mi_scores))
feature_mi_scores.sort(key=lambda x: x[1], reverse=True)

In [11]:
print(feature_mi_scores)

[('residual sugar', 0.7099142254240514), ('volatile acidity', 0.6129322545833249), ('chlorides', 0.5692674129951456), ('alcohol', 0.15419570629930046), ('density', 0.1523761157913519), ('pH', 0.12521650488210856), ('free sulfur dioxide', 0.10298356730827596), ('sulphates', 0.07844744445197982), ('total sulfur dioxide', 0.07274734558005758), ('fixed acidity', 0.05812778798633822), ('citric acid', 0.014222772872591882)]


NOTE : Variables derived at the start were using the mi scores

Splitting into training and test set

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=0)

Feature Scaling

In [13]:
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
sc_y=StandardScaler()
X_train=sc_x.fit_transform(X_train)
Y_train=sc_y.fit_transform(Y_train.reshape(-1,1))

Training the Random Forest Model on training set

In [14]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 500, random_state = 0)
regressor.fit(X_train,Y_train)

<ipython-input-14-4388757ce2e2>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train,Y_train)


RandomForestRegressor(n_estimators=500, random_state=0)

NOTE : Random Forest provides best results after trying with other regression models

Predicting the Test set

In [15]:
Y_pred=sc_y.inverse_transform(regressor.predict(sc_x.transform(X_test)).reshape(-1,1))

In [16]:
print(Y_pred.reshape(1,-1))

[[5.996 6.004 6.016 ... 6.    7.    5.004]]


In [17]:
print(Y_test)

[6 6 6 ... 6 7 5]


Testing the score of the model

In [18]:
from sklearn.metrics import r2_score
r2_score(Y_test,Y_pred.reshape(-1,1))

0.9789769871482509